# Identify affected buildings

In this notebook we show how to identify the affected buildings in a floodmap. We will use GeoScape buildings in ESRI ShapeFile provided by NEMA which are available [here](https://drive.google.com/drive/folders/1NH5Pu71jBxmchTKSeYyj7VF26WXLvowN?usp=share_link). For the purpose of this demonstration we will use the floodmap over Lismore aggregated between the 27th of March to the 15th April of 2022.

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/gonzalo/Downloads/nema-floodmapper-2022.json"

In [ ]:
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] missing $GOOGLE_APPLICATION_CREDENTIALS!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "

## Load and plot the floodmap

In [ ]:
import matplotlib.colors
from ml4floods.data import utils
import geopandas as gpd


COLORS = {
    'cloud': "gray",
    'flood_trace': "turquoise",
    'water': "blue"
}


floodmap = utils.read_geojson_from_gcp("gs://ml4floods_nema/0_DEV/1_Staging/operational/EMSR570/AUTOAOI3011/pre_post_products/postflood_2022-03-27_2022-04-15.geojson")
area_imaged = floodmap[floodmap["class"] == "area_imaged"].copy()
floodmap = floodmap[floodmap["class"] != "area_imaged"].copy()

categories = floodmap["class"].unique()
cmap = matplotlib.colors.ListedColormap([COLORS[b] for b in categories])

m = floodmap.explore(column="class",cmap=cmap,categories=categories)

# Plot in red the area of interest
m = area_imaged.explore(m=m,color="red",style_kwds={"fill":False})
m

## Convert floodmap to CRS EPSG:7844

This step is needed because GeoScape buildings are saved in this CRS. In addition we subset the floodmap `GeoDataFrame` object to keep only the `water` and `flood_trace` polygons.

In [ ]:
floodmap_crs7844 = floodmap.to_crs("EPSG:7844")
area_imaged_crs7844 = area_imaged.to_crs("EPSG:7844")
floodmap_crs7844_innundated = floodmap_crs7844[floodmap_crs7844["class"].isin(["water","flood_trace"])]

## Read buildings over the area of interest

In the next cell we read the GeoScape buildings over the area of interest. For this we use the `read_file` function of `geopandas` and specify a bounding box (`bbox` argument) to read. This makes the reading of the buildings much faster (as we only load the buildings in this bounding box instead of all buildings in NSW).

In [ ]:
%%time
# https://drive.google.com/drive/folders/1NH5Pu71jBxmchTKSeYyj7VF26WXLvowN?usp=share_link
path_buildings = "/home/gonzalo/Downloads/Buildings/Buildings JUNE 2022/Standard/nsw_buildings.shp"
buildings_in_flood_area = gpd.read_file(path_buildings,bbox=tuple(area_imaged_crs7844.iloc[0].geometry.bounds))
buildings_in_flood_area

## Compute affected buildings

In the next cell we intersect the buildings and the floodmap. We add a column in the buildings `GeoDataFrame` the indicates if the building intersect the floodmap (i.e. if it has been affected by the flooding).

In [ ]:
%%time

# Add column whether the building has been affected by the flood
buildings_in_flood_area["affected"] = buildings_in_flood_area.geometry.apply(lambda x: "yes" if floodmap_crs7844_innundated.intersects(x).any() else "no")

## Display affected buildings

In the next cell we show in red the affected buildings, we also show in gray the non-affected buildings.

In [ ]:
categories_buildings = ["yes","no"]
cmap_buildings = matplotlib.colors.ListedColormap(["#DD0000","#888888"])

m = buildings_in_flood_area[["geometry","affected","CAPT_DATE","AREA","GRD_ELEV"]].explore(column="affected",cmap=cmap_buildings,categories=categories_buildings)

# Plot in red the area of interest
m = area_imaged.explore(m=m,color="red",style_kwds={"fill":False})
m

## Display affected buildings and floodmap together

We could also display together the affected buildings and the floodmap.

In [ ]:
m = buildings_in_flood_area[["geometry","affected","CAPT_DATE","AREA","GRD_ELEV"]].explore(column="affected",cmap=cmap_buildings,categories=categories_buildings)

# Plot in red the area of interest
m = area_imaged.explore(m=m,color="red",style_kwds={"fill":False})
m = floodmap.explore(column="class",cmap=cmap,categories=categories,style_kwds={"fillOpacity":0.25},m=m)

m

## Statistics of affected buildings

In [ ]:
buildings_in_flood_area_affected = buildings_in_flood_area[buildings_in_flood_area.affected == "yes"]

print(f"There are {buildings_in_flood_area_affected.shape[0]} affected buildings")
print(f"Area of the affected buildings: {buildings_in_flood_area_affected.AREA.sum()} m²")